In [ ]:
!pip install anywidget

In [ ]:
import anywidget
import traitlets

ESM = """
import * as p from 'https://unpkg.com/htm/preact/standalone.module.js'

// HOOKS
let WidgetModelContext = p.createContext(undefined);

function useModelState(name) {
    let model = p.useContext(WidgetModelContext);
    let [state, setState] = p.useState(model.get(name));
    p.useEffect(() => {
        let event = `change:${name}`;
        function callback(model) {
            setState(model.get(name));
        }
        model.on(event, callback);
        return () => {
            model.off(event, callback);
        }
    }, [model, name])
    return [state, (val, options) => {
        model.set(name, val, options);
        model.save_changes();
    }];
}

function Counter() {
    let [count, setCount] = useModelState("count");
    return p.html`<div style=${{ display: "grid", gridAutoFlow: "column", textAlign: "center" }}>
        <button onClick=${() => setCount(count - 1)}> - </button>
        <p>${count}</p>
        <button onClick=${() => setCount(count + 1)}> + </button>
    </div>`;
    
}

export function render(view) {
    p.render(p.html`<${WidgetModelContext.Provider} value=${view.model}>
        <${Counter} />
    </${WidgetModelContext.Provider}>`, view.el);
}
"""

class CounterWidget(anywidget.AnyWidget):
    _esm = ESM
    count = traitlets.Int(0).tag(sync=True)

In [ ]:
counter = CounterWidget()
counter

In [ ]:
counter.count = 5